In [5]:
import requests
import json
import pandas as pd
import time
import os
from datetime import datetime
import psycopg2

from dotenv import load_dotenv

load_dotenv()

TINY_TOKEN = os.getenv("TINY_TOKEN")
HOST = os.getenv("HOST")
POSTGRES_DB = os.getenv("POSTGRES_DB")
POSTGRES_USER = os.getenv("POSTGRES_USER")
POSTGRES_PASSWORD = os.getenv("POSTGRES_PASSWORD")

# Informações de conexão com o banco de dados PostgreSQL
db_config = {
    "host": HOST,
    "database": POSTGRES_DB,
    "user": POSTGRES_USER,
    "password": POSTGRES_PASSWORD,
}

# Configurações da API Tiny

token = TINY_TOKEN
formato = 'JSON'

In [19]:
# Carregando IDs de produtos

conn = psycopg2.connect(**db_config) # Conecta ao banco de dados usando db_config

# query = "select * from tiny_fulfillment;"
query = "select tiny_id from tiny_fulfillment;"

# tiny_id_df = pd.read_sql(query, db_config)
df_tiny_id_original = pd.read_sql_query(query, conn)

conn.close()

df_tiny_id_original.shape

/tmp/ipykernel_103058/1161420276.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_tiny_id_original = pd.read_sql_query(query, conn)


(872, 1)

In [20]:
df_tiny_id_original

,tiny_id
0,735947207
1,747848159
2,698412654
3,749528560
4,749530945
...,...
867,565665071
868,509636083
869,729127557
870,729130774


In [21]:
# Salve o DataFrame original
df_tiny_id = df_tiny_id_original.copy()

# Agora, df conterá apenas a coluna especificada
df_tiny_id.shape

(872, 1)

In [22]:
# Remova as linhas com valores NaN - Linha 378
df_tiny_id_no_nan = df_tiny_id.dropna(subset=['tiny_id'])

df_tiny_id_no_nan.shape

(872, 1)

In [23]:
# Verifica se há duplicatas
duplicatas = df_tiny_id_no_nan['tiny_id'].duplicated()
numero_de_duplicatas = duplicatas.sum()

print(f'Número de duplicatas: {numero_de_duplicatas}')

# # Encontre as duplicatas no DataFrame
duplicatas = df_tiny_id_no_nan[df_tiny_id_no_nan['tiny_id'].duplicated(keep=False)]

# Exiba as próprias duplicatas
duplicatas

Número de duplicatas: 391


,tiny_id
0,735947207
1,747848159
2,698412654
9,749530553
10,749530597
...,...
865,596827212
866,565663877
867,565665071
869,729127557


In [24]:
# Remove as duplicatas e atualiza o DataFrame
df_tiny_id_no_dup = df_tiny_id_no_nan.drop_duplicates()

df_tiny_id_no_dup.shape

(481, 1)

In [43]:
# Requisitando produtos à API Tiny

url = 'https://api.tiny.com.br/api2/produto.obter.estoque.php'


id = '759523162'

def enviarREST(url, data, optional_headers=None):
    headers = optional_headers if optional_headers is not None else {}

    response = requests.post(url, data=data, headers=headers)

    if response.status_code != 200:
        raise Exception(f"Problema com {url}, Status Code: {response.status_code}")

    return response.text


data = {'token': token, 'id': id, 'formato': formato}
response = enviarREST(url, data)
response

'{"retorno":{"status_processamento":"3","status":"OK","produto":{"id":"759523162","nome":"Capa Ukulele Soprano Imperme\\u00e1vel Super Luxo","codigo":"A90","unidade":"UN","saldo":15,"saldoReservado":0,"depositos":[{"deposito":{"nome":"Cristiano","desconsiderar":"N","saldo":10,"empresa":"mcenter168"}},{"deposito":{"nome":"M A Bueno","desconsiderar":"N","saldo":10,"empresa":"mcenter168"}},{"deposito":{"nome":"M A Lang","desconsiderar":"N","saldo":0,"empresa":"mcenter168"}},{"deposito":{"nome":"Mundo das Teclas","desconsiderar":"N","saldo":0,"empresa":"mcenter168"}},{"deposito":{"nome":"Musical Filial","desconsiderar":"N","saldo":0,"empresa":"mcenter168"}},{"deposito":{"nome":"Musical Matriz","desconsiderar":"N","saldo":-5,"empresa":"mcenter168"}},{"deposito":{"nome":"Talyta","desconsiderar":"N","saldo":0,"empresa":"mcenter168"}}]}}}'

In [25]:
# Requisitando produtos à API Tiny

url = 'https://api.tiny.com.br/api2/produto.obter.estoque.php'

# df_tiny_id_no_dup = df_tiny_id_no_dup.head(3)
#df_tiny_id = pd.read_csv('tiny_id_no_dup.csv')
df_tiny_id = df_tiny_id_no_dup

responses = []  # Lista para armazenar os resultados
num = 0

def enviarREST(url, data, optional_headers=None):
    headers = optional_headers if optional_headers is not None else {}

    response = requests.post(url, data=data, headers=headers)

    if response.status_code != 200:
        raise Exception(f"Problema com {url}, Status Code: {response.status_code}")

    return response.text


for id in df_tiny_id['tiny_id']:
    print(f'Buscando dados de {id}')
    num +=1
    print(f'Loop: {num}')
    data = {'token': token, 'id': id, 'formato': formato}
    response = enviarREST(url, data)
    responses.append(response)

    # Verifica se é múltiplo de 50 para aguardar 1min a cada 50 requisições
    if num % 50 == 0:
        print("Esperando 1 minuto...")
        time.sleep(60)  # Pausa por 1 minuto

# Registra o tempo depois da execução
end_time = time.time()

# Calcula o tempo decorrido
elapsed_time = end_time - start_time

print(f"\n\nTempo :decorrido {elapsed_time / 60} minutos")
print(f"Total de resultados encontrados: {len(responses)}")

Buscando dados de 735947207
Loop: 1
Buscando dados de 747848159
Loop: 2
Buscando dados de 698412654
Loop: 3
Buscando dados de 749528560
Loop: 4
Buscando dados de 749530945
Loop: 5
Buscando dados de 749530672
Loop: 6
Buscando dados de 749528632
Loop: 7
Buscando dados de 749528951
Loop: 8
Buscando dados de 749528930
Loop: 9
Buscando dados de 749530553
Loop: 10
Buscando dados de 749530597
Loop: 11
Buscando dados de 749529078
Loop: 12
Buscando dados de 749530521
Loop: 13
Buscando dados de 749530626
Loop: 14
Buscando dados de 746248995
Loop: 15
Buscando dados de 746248920
Loop: 16
Buscando dados de 746249012
Loop: 17
Buscando dados de 606234702
Loop: 18
Buscando dados de 695806136
Loop: 19
Buscando dados de 735893980
Loop: 20
Buscando dados de 506535214
Loop: 21
Buscando dados de 692720767
Loop: 22
Buscando dados de 680629622
Loop: 23
Buscando dados de 680629593
Loop: 24
Buscando dados de 680629663
Loop: 25
Buscando dados de 680629606
Loop: 26
Buscando dados de 699653695
Loop: 27
Buscando d

In [26]:
responses

['{"retorno":{"status_processamento":"3","status":"OK","produto":{"id":"735947207","nome":"BAQUETA LIVERPOOL NIRVANA ECOBIO 5B MADEIRA NI5BM","codigo":"7897937421021","unidade":"PR","saldo":14035,"saldoReservado":163,"depositos":[{"deposito":{"nome":"Cristiano","desconsiderar":"N","saldo":0,"empresa":"mcenter168"}},{"deposito":{"nome":"M A Bueno","desconsiderar":"N","saldo":977,"empresa":"mcenter168"}},{"deposito":{"nome":"M A Lang","desconsiderar":"N","saldo":0,"empresa":"mcenter168"}},{"deposito":{"nome":"Mundo das Teclas","desconsiderar":"N","saldo":0,"empresa":"mcenter168"}},{"deposito":{"nome":"Musical Filial","desconsiderar":"N","saldo":0,"empresa":"mcenter168"}},{"deposito":{"nome":"Musical Matriz","desconsiderar":"N","saldo":1718,"empresa":"mcenter168"}},{"deposito":{"nome":"Talyta","desconsiderar":"N","saldo":0,"empresa":"mcenter168"}},{"deposito":{"nome":"Geral","desconsiderar":"N","saldo":11340,"empresa":"cpmusical"}}]}}}',
 '{"retorno":{"status_processamento":"3","status":"

In [27]:
# Gravando respostas em txt
datetime_now = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

file_path = f"../Data/Output/tiny_estoque_json.txt"

with open(file_path, 'w') as file:
    for line in responses:
        file.write(line + '\n')  # Adicione uma quebra de linha

print(f'Os dados foram gravados no arquivo: {file_path}')

Os dados foram gravados no arquivo: ../Data/Output/tiny_estoque_json_2023-11-22_14-11-11.txt


In [28]:
# Verificandos e há erros de processamento
text_search1 = '"status_processamento":"1"'
text_search2 = '"status_processamento":"2"'

lines_search1 = []
lines_search2 = []

with open(file_path, 'r') as file:
    for n_line, line in enumerate(file, start=1):
        if text_search1 in line:
            lines_search1.append(n_line)
        if text_search2 in line:
            lines_search2.append(n_line)

if lines_search1 and lines_search2:
    print(f'"status_processamento":"1" foi encontrado nas linhas: {lines_search1}')
    print(f'"status_processamento":"2" foi encontrado nas linhas: {lines_search2}')
elif lines_search1:
    print(f'"status_processamento":"1" foi encontrado nas linhas: {lines_search1}')
    print('"status_processamento":"2" não foi encontrado em nenhuma linha.')
elif lines_search2:
    print(f'"status_processamento":"2" foi encontrado nas linhas: {lines_search2}')
    print('"status_processamento":"1" não foi encontrado em nenhuma linha.')
else:
    print('Não foi verificado erro de processamento"')


Não foi verificado erro de processamento"


In [44]:
# lista de JSONs
lista_json = responses

# Inicialize o DataFrame vazio
tiny_estoque_df = pd.DataFrame()

# Processar cada JSON na lista
for json_str in lista_json:
    # Transforme a string JSON em um objeto Python
    json_data = json.loads(json_str)

    # Extrair a parte "produto" do JSON
    produto = json_data["retorno"]["produto"]
    depositos = produto['depositos']

    # Verifique se a lista de depósitos está presente
    if depositos:
        # Crie um DataFrame temporário a partir da lista de depósitos
        temp_df = pd.json_normalize(depositos)

        # Adicione as colunas do nível "produto" ao DataFrame temporário
        for key, value in produto.items():
            temp_df[key] = value

        # Concatene o DataFrame temporário ao DataFrame principal
        # produto_df = pd.concat([produto_df, temp_df], ignore_index=True)
        tiny_estoque_df = pd.concat([tiny_estoque_df, temp_df], ignore_index=True)

# Reordene as colunas, se necessário
col_order = [
    'id', 'nome', 'codigo', 'unidade', 'saldo', 'saldoReservado',
    'deposito.nome', 'deposito.desconsiderar', 'deposito.saldo', 'deposito.empresa'
]
tiny_estoque_df = tiny_estoque_df[col_order]

# O DataFrame 
tiny_estoque_df.head(1)
tiny_estoque_df.shape


(3445, 10)

In [39]:
tiny_estoque_df.sample(5)

,id,nome,codigo,unidade,saldo,saldoReservado,deposito.nome,deposito.desconsiderar,deposito.saldo,deposito.empresa
699,732040398,ENC. 013-056 P/ VIOLAO ACO EARTHWOOD TENSAO ME...,749699121442,PC,16,0,M A Bueno,N,0,mcenter168
836,729130552,CORREIA INST CORDA DG 04 PRETO BASSO,7898942136313,UN,123,6,Musical Matriz,N,0,mcenter168
1459,693265016,GUITARRA PHX MARVEL CAPITAO AMERICA KIDS GMC-K2,7899548663340,UN,0,0,Musical Filial,N,0,mcenter168
2449,558644728,Suporte Para Tuba Hercules DS552B 5313,7897626853133,Un,3,0,Musical Matriz,N,0,mcenter168
3238,748837888,PALHETA P/SAX ALTO TRADICIONAL 3 SR213 VANDORE...,7897626822139,UN,114,10,Musical Filial,N,20,mcenter168


In [30]:
# Verifica se há duplicatas
duplicatas = tiny_estoque_df.duplicated()
total_duplicatas = duplicatas.sum() 

print(f'Número de duplicatas: {total_duplicatas}')

# Encontra duplicatas no DataFrame
duplicatas = tiny_estoque_df[tiny_estoque_df.duplicated(keep=False)]

# Exiba duplicatas
duplicatas['id']

l_duplicatas = duplicatas[duplicatas['id'] == '565663877']
l_duplicatas.shape

Número de duplicatas: 0


(0, 10)

In [31]:
tiny_estoque_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3445 entries, 0 to 3444
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   id                      3445 non-null   object
 1   nome                    3445 non-null   object
 2   codigo                  3445 non-null   object
 3   unidade                 3445 non-null   object
 4   saldo                   3445 non-null   int64 
 5   saldoReservado          3445 non-null   int64 
 6   deposito.nome           3445 non-null   object
 7   deposito.desconsiderar  3445 non-null   object
 8   deposito.saldo          3445 non-null   int64 
 9   deposito.empresa        3445 non-null   object
dtypes: int64(3), object(7)
memory usage: 269.3+ KB


In [32]:
unique_values = tiny_estoque_df['deposito.empresa'].unique()
print(unique_values)

['mcenter168' 'cpmusical']


In [33]:
# Remove linhas referentes a empresa cpmusical
remove_val = 'cpmusical'

# Crie um novo DataFrame sem as linhas que contêm o valor específico
tiny_estoque_df = tiny_estoque_df[tiny_estoque_df['deposito.empresa'] != remove_val]

# Agora novo_df contém todas as linhas onde 'deposito.empresa' não é igual a 'EmpresaX'
tiny_estoque_df.shape

(3367, 10)

In [40]:
tiny_estoque_df.sample(5)

,id,nome,codigo,unidade,saldo,saldoReservado,deposito.nome,deposito.desconsiderar,deposito.saldo,deposito.empresa
1382,640788247,FLAUTA DOCE PHX BARROCA ROSA P9,7899548630113,UN,0,0,Musical Matriz,N,0,mcenter168
2877,736122790,Ukulele Acustico Honu cordas de Nylon Tagima M...,7898651948917,PC,0,0,M A Lang,N,0,mcenter168
1232,742019381,ENC. 012 HEAVY P/GUITARRA 12152 ELIXIR 5723,7897626857230,PC,10,0,Cristiano,N,0,mcenter168
818,729132044,CORREIA INST CORDA VT SL 60 RELAX BRANCO BASSO,7898529535850,UN,107,1,M A Bueno,N,0,mcenter168
2609,579932492,Tantan Reto 12 pol 50 cm Aluminio c/ Aro Croma...,7897626882522,Un,588,13,Mundo das Teclas,N,33,mcenter168


### Salvando na base de dados

In [34]:
tiny_estoque_df.columns

Index(['id', 'nome', 'codigo', 'unidade', 'saldo', 'saldoReservado',
       'deposito.nome', 'deposito.desconsiderar', 'deposito.saldo',
       'deposito.empresa'],
      dtype='object')

In [42]:
try:
    # Tenta fazer a conexão com o banco de dados
    conn = psycopg2.connect(**db_config)

    # Cria um cursor
    cursor = conn.cursor()

    n = 1

    # Itera sobre as linhas do DataFrame e insere os dados na tabela "tiny_stock"
    for index, row in tiny_estoque_df.iterrows():
        print(f'Loop nº: {n}/{tiny_estoque_df.shape[0]}')
        query = """
        INSERT INTO tiny_stock_hist (tiny_id, nome, sku_tiny, unidade, saldo_reservado, deposito_nome, deposito_desconsiderar,
        deposito_saldo, deposito_empresa)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s);
        """

        values = (
            row['id'],
            row['nome'],
            row['codigo'],
            row['unidade'],
            row['saldoReservado'],
            row['deposito.nome'],
            row['deposito.desconsiderar'],
            row['deposito.saldo'],
            row['deposito.empresa'],
        )

        n += 1

        # Tenta executar a query
        cursor.execute(query, values)

    # Faz o commit para salvar as alterações no banco de dados
    conn.commit()
    print('Produtos adicionados ao Banco de dados.')

except psycopg2.Error as e:
    # Em caso de erro específico do psycopg2, exibe a mensagem de erro
    print(f"Erro do psycopg2: {e}")

except Exception as e:
    # Em caso de outros erros, exibe a mensagem de erro
    print(f"Erro: {e}")

finally:
    # Fecha o cursor e a conexão, independentemente de ter ocorrido um erro ou não
    if cursor is not None:
        cursor.close()
    if conn is not None:
        conn.close()


Loop nº: 1/3367
Loop nº: 2/3367
Loop nº: 3/3367
Loop nº: 4/3367
Loop nº: 5/3367
Loop nº: 6/3367
Loop nº: 7/3367
Loop nº: 8/3367
Loop nº: 9/3367
Loop nº: 10/3367
Loop nº: 11/3367
Loop nº: 12/3367
Loop nº: 13/3367
Loop nº: 14/3367
Loop nº: 15/3367
Loop nº: 16/3367
Loop nº: 17/3367
Loop nº: 18/3367
Loop nº: 19/3367
Loop nº: 20/3367
Loop nº: 21/3367
Loop nº: 22/3367
Loop nº: 23/3367
Loop nº: 24/3367
Loop nº: 25/3367
Loop nº: 26/3367
Loop nº: 27/3367
Loop nº: 28/3367
Loop nº: 29/3367
Loop nº: 30/3367
Loop nº: 31/3367
Loop nº: 32/3367
Loop nº: 33/3367
Loop nº: 34/3367
Loop nº: 35/3367
Loop nº: 36/3367
Loop nº: 37/3367
Loop nº: 38/3367
Loop nº: 39/3367
Loop nº: 40/3367
Loop nº: 41/3367
Loop nº: 42/3367
Loop nº: 43/3367
Loop nº: 44/3367
Loop nº: 45/3367
Loop nº: 46/3367
Loop nº: 47/3367
Loop nº: 48/3367
Loop nº: 49/3367
Loop nº: 50/3367
Loop nº: 51/3367
Loop nº: 52/3367
Loop nº: 53/3367
Loop nº: 54/3367
Loop nº: 55/3367
Loop nº: 56/3367
Loop nº: 57/3367
Loop nº: 58/3367
Loop nº: 59/3367
Loop n